In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Example

In [3]:
# example
from transformers import Wav2Vec2FeatureExtractor, UniSpeechSatForXVector, AutoProcessor, AutoModelForAudioXVector
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

# feature_extractor = AutoProcessor.from_pretrained("microsoft/unispeech-sat-base-plus-sv")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/unispeech-sat-base-plus-sv')
# model = UniSpeechSatForXVector.from_pretrained('microsoft/unispeech-sat-base-plus-sv')
model = AutoModelForAudioXVector.from_pretrained("microsoft/unispeech-sat-base-plus-sv")


Found cached dataset librispeech_asr_demo (/Users/amanmibra/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_demo/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


In [4]:
# test_dataset = load_dataset("librispeech_asr", split="test")

In [5]:
dataset

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [6]:
dataset[:2]['audio']

[{'path': '/Users/amanmibra/.cache/huggingface/datasets/downloads/extracted/a30c5847976d1177bc2124e158ae601757871524213cd21d21be61ac5a29196e/dev_clean/1272/128104/1272-128104-0000.flac',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
         0.0010376 ]),
  'sampling_rate': 16000},
 {'path': '/Users/amanmibra/.cache/huggingface/datasets/downloads/extracted/a30c5847976d1177bc2124e158ae601757871524213cd21d21be61ac5a29196e/dev_clean/1272/128104/1272-128104-0001.flac',
  'array': array([-1.52587891e-04, -9.15527344e-05, -1.83105469e-04, ...,
          9.76562500e-04,  9.46044922e-04, -4.88281250e-04]),
  'sampling_rate': 16000}]

In [7]:
speechs = [ data["array"] for data in dataset[:]["audio"]]

In [8]:
speechs[12].shape[0]/16000

5.38

In [9]:
len(speechs)

73

In [10]:
op_speeches = [data["audio"]["array"] for data in dataset if data["speaker_id"] != dataset[0]["speaker_id"]]
dataset[0]

{'file': '/Users/amanmibra/.cache/huggingface/datasets/downloads/extracted/a30c5847976d1177bc2124e158ae601757871524213cd21d21be61ac5a29196e/dev_clean/1272/128104/1272-128104-0000.flac',
 'audio': {'path': '/Users/amanmibra/.cache/huggingface/datasets/downloads/extracted/a30c5847976d1177bc2124e158ae601757871524213cd21d21be61ac5a29196e/dev_clean/1272/128104/1272-128104-0000.flac',
  'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
         0.0010376 ]),
  'sampling_rate': 16000},
 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL',
 'speaker_id': 1272,
 'chapter_id': 128104,
 'id': '1272-128104-0000'}

In [11]:
# audio files are decoded on the fly
input_speeches = [speechs[0], speechs[-11]]
inputs = feature_extractor(input_speeches, return_tensors="pt", padding=True, sampling_rate=16000)
embeddings = model(**inputs).embeddings
embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()

In [12]:
# the resulting embeddings can be used for cosine similarity-based retrieval
cosine_sim = torch.nn.CosineSimilarity(dim=-1)
similarity = cosine_sim(embeddings[0], embeddings[1])
threshold = 0.89  # the optimal threshold is dataset-dependent
if similarity < threshold:
    print("Speakers are not the same!")
print(f"{similarity}")

0.9834722280502319


# Comparing Audio

In [13]:
import os
import torchaudio
from IPython.display import Audio

In [37]:
AMAN = os.path.join('..', 'data', 'test', 'aman', 'aman_1.wav')
IMRAN = os.path.join('..', 'data', 'test', 'imran', 'imran_1.wav')
LABIB = os.path.join('..', 'data', 'test', 'labib', 'labib_1.wav')
AMAN2 = os.path.join('..', 'data', 'test', 'aman', 'aman_1.wav')
IMRAN2 = os.path.join('..', 'data', 'test', 'imran', 'imran_2.wav')
LABIB2 = os.path.join('..', 'data', 'test', 'labib', 'labib_2.wav')

# for s in speechs:
#     display(Audio(s, rate=16000))
aman, sr = torchaudio.load(AMAN)
imran, sr = torchaudio.load(IMRAN)
labib, sr = torchaudio.load(LABIB)
aman2, sr = torchaudio.load(AMAN2)
imran2, sr = torchaudio.load(IMRAN2)
labib2, sr = torchaudio.load(LABIB2)


In [46]:
for f in [AMAN, AMAN2, IMRAN, IMRAN2, LABIB, LABIB2]:
    wav, sr = torchaudio.load(f)
    display(Audio(wav, rate=sr))
    type(wav)

In [39]:
resampler = torchaudio.transforms.Resample(48000, 16000)
wav = resampler(wav)

In [40]:
display(Audio(speechs[0], rate=16000))
type(speechs[0])
speech_1 = speechs[-1]

In [41]:
display(Audio(wav, rate=16000))
type(wav)
speech_0 = wav.squeeze(0).numpy()

In [20]:
test_speechs = [speech_0] + speechs
inputs = feature_extractor(test_speechs, return_tensors="pt", padding='longest', sampling_rate=16000)
inputs

{'input_values': tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.3804e-03,  2.0752e-03,  1.9836e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.5259e-04, -9.1553e-05, -1.8311e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        ...,
        [-3.0518e-05, -1.8311e-04, -2.1362e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.9673e-04, -3.0518e-05,  8.8501e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.8311e-03, -2.1362e-04, -6.1035e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [21]:
import time
now = time.time()

embeddings = model(input_values=inputs['input_values'][:10], attention_mask=inputs['attention_mask'][:10]).embeddings
embeddings = torch.nn.functional.normalize(embeddings, dim=-1).cpu()
print(f"{time.time() - now}s")
embeddings

31.549786806106567s


tensor([[-0.0208, -0.0214, -0.0257,  ..., -0.0246, -0.0194, -0.0213],
        [-0.0226, -0.0234, -0.0212,  ..., -0.0270, -0.0185, -0.0185],
        [-0.0178, -0.0236, -0.0260,  ..., -0.0262, -0.0185, -0.0210],
        ...,
        [-0.0218, -0.0231, -0.0254,  ..., -0.0242, -0.0199, -0.0211],
        [-0.0216, -0.0253, -0.0248,  ..., -0.0265, -0.0183, -0.0213],
        [-0.0221, -0.0286, -0.0264,  ..., -0.0242, -0.0188, -0.0229]],
       grad_fn=<DivBackward0>)

In [111]:
def compare(x, y):
    cosine_sim = torch.nn.CosineSimilarity(dim=-1)
    similarity = cosine_sim(x, y)
    threshold = 0.89  # the optimal threshold is dataset-dependent
    if similarity < threshold:
        print("Speakers are not the same!")
    print(f"{similarity}")

def compare2(embs1, embs2):
    X = embs1 / torch.linalg.norm(embs1)
    Y = embs2 / torch.linalg.norm(embs2)
    
    # Score
    similarity_score = torch.dot(X, Y) / ((torch.dot(X, X) * torch.dot(Y, Y)) ** 0.5)
    similarity_score = (similarity_score + 1) / 2
    print(similarity_score)
    return similarity_score

In [42]:
def resample(wav):
    resampler = torchaudio.transforms.Resample(48000, 16000)
    return resampler(wav).numpy()

aman = resample(aman.squeeze(0))
imran = resample(imran.squeeze(0))
labib = resample(labib.squeeze(0))
aman2 = resample(aman2.squeeze(0))
imran2 = resample(imran2.squeeze(0))
labib2 = resample(labib2.squeeze(0))

features = [
    aman, aman2, imran, imran2, labib, labib2
]


features = feature_extractor(features, return_tensors="pt", padding=True, sampling_rate=16000)
features

{'input_values': tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 6.4122e-05, 4.2548e-05,
         8.1797e-06],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [43]:
output = model(**features)

In [47]:
output.keys()

odict_keys(['logits', 'embeddings', 'hidden_states'])

In [48]:
output.logits

tensor([[0.5125, 0.4476, 0.2650,  ..., 1.1553, 0.2939, 0.8148],
        [0.5125, 0.4476, 0.2650,  ..., 1.1553, 0.2939, 0.8148],
        [0.6040, 0.5570, 0.3221,  ..., 1.4030, 0.5556, 0.9059],
        [0.5743, 0.6571, 0.2845,  ..., 1.0004, 0.3817, 0.8875],
        [0.4483, 0.5341, 0.2780,  ..., 1.3070, 0.5197, 0.8530],
        [0.3154, 0.4677, 0.2395,  ..., 0.9950, 0.4153, 0.7480]],
       grad_fn=<AddmmBackward0>)

In [49]:
emb = output.embeddings
emb

tensor([[-0.0896, -0.0916, -0.1316,  ..., -0.1086, -0.0879, -0.0967],
        [-0.0896, -0.0916, -0.1316,  ..., -0.1086, -0.0879, -0.0967],
        [-0.1281, -0.1042, -0.1486,  ..., -0.1319, -0.0903, -0.0850],
        [-0.1031, -0.1013, -0.1439,  ..., -0.1227, -0.0851, -0.0763],
        [-0.1044, -0.1043, -0.1051,  ..., -0.1266, -0.0954, -0.0919],
        [-0.0833, -0.0844, -0.0854,  ..., -0.1045, -0.0808, -0.0805]],
       grad_fn=<AddmmBackward0>)

In [50]:
emb = torch.nn.functional.normalize(emb, dim=-1)
emb

tensor([[-0.0211, -0.0215, -0.0309,  ..., -0.0255, -0.0207, -0.0227],
        [-0.0211, -0.0215, -0.0309,  ..., -0.0255, -0.0207, -0.0227],
        [-0.0270, -0.0219, -0.0313,  ..., -0.0278, -0.0190, -0.0179],
        [-0.0230, -0.0226, -0.0321,  ..., -0.0274, -0.0190, -0.0170],
        [-0.0236, -0.0235, -0.0237,  ..., -0.0285, -0.0215, -0.0207],
        [-0.0223, -0.0226, -0.0229,  ..., -0.0280, -0.0216, -0.0215]],
       grad_fn=<DivBackward0>)

In [60]:
compare(emb[0], emb[4])

0.921717643737793


In [30]:
for s in [aman, labib, imran]:
    display(Audio(s, rate=16000))

In [65]:
AISF_AMAN = os.path.join('..', 'data', 'aisf', 'test', 'aman', 'aman_1.wav')
wav, sr = torchaudio.load(AISF_AMAN)
wav = resample(wav.squeeze(0))

aisf_emb = model(**feature_extractor(wav, return_tensors="pt", padding=True, sampling_rate=16000)).embeddings

In [79]:
compare(aisf_emb[0], emb[4])
print(aisf_emb[0].shape)
print(emb[0].shape)

0.9013770222663879
torch.Size([512])
torch.Size([512])


In [32]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": false,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [ ]:
model

In [ ]:
model.tdnn

In [81]:
import nemo.collections.asr as nemo_asr
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")


/Users/amanmibra/anaconda3/envs/void/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/amanmibra/anaconda3/envs/void/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/amanmibra/anaconda3/envs/void/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <DAEF2BE1-BBB4-3005-8003-63A504CDB9D9> /Users/amanmibra/anaconda3/envs/void/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
[NeMo W 2023-05-26 10:05:59 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-05-26 10:06:00 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.

[NeMo I 2023-05-26 10:06:01 features:287] PADDING: 16
[NeMo I 2023-05-26 10:06:01 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /Users/amanmibra/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.


In [87]:
i_emb = speaker_model.get_embedding(IMRAN)

In [102]:
a_emb = speaker_model.get_embedding(AISF_AMAN)

In [94]:
compare(a_emb[0], a_emb[0])

1.0


In [105]:
emb2 = []
for f in [AMAN, AMAN2, IMRAN, IMRAN2, LABIB, LABIB2]:
    emb2.append(speaker_model.get_embedding(f).squeeze())

In [103]:
compare(emb2[0], a_emb)

Speakers are not the same!
tensor([0.2430])


In [84]:
speaker_model.verify_speakers(AMAN, LABIB)

[NeMo I 2023-05-26 10:06:04 label_models:509]  two audio files are from different speakers


False

In [86]:
speaker_model.verify_speakers(AMAN, AISF_AMAN)

[NeMo I 2023-05-26 10:06:19 label_models:509]  two audio files are from different speakers


False

In [80]:
AMAN2 = os.path.join('..', 'data', 'test', 'aman', 'aman_2.wav')
speaker_model.verify_speakers(AMAN, LABIB)

NameError: name 'speaker_model' is not defined

In [112]:
for e in emb2[1:]:
    compare2(emb2[0], e)

tensor(0.8705)
tensor(0.6334)
tensor(0.6163)
tensor(0.5945)
tensor(0.6159)


In [115]:
compare2(emb2[0], a_emb.squeeze())

tensor(0.6215)


tensor(0.6215)